In [1]:
%load_ext autoreload
%autoreload 2
    
%load_ext tensorboard
import sys, os
sys.path.append('../paraphrase/')
from paraphraser_args import ModelArguments, DataTrainingArguments, TrainingArguments
from paraphraser_dataloader import load_dataset_pseudo, load_dataset_pseudo_binary
from paraphraser_trainer import ParaphraserTrainer
from transformers import AutoTokenizer, AutoModelWithLMHead, HfArgumentParser

In [2]:
# data_dir = "../data/paraphrase/paranmt_filtered"
# model_name = "t5-small"
# model_nick = "t5_paraphrase"
# output_dir = "../models/"
# epochs = "12"
# train_batch_size = "16"
# eval_batch_size = "16"
# save_log_steps = "400"

# parser = HfArgumentParser((ModelArguments, DataTrainingArguments, TrainingArguments))
# model_args, data_args, training_args = parser.parse_args_into_dataclasses([
#     "--model_name_or_path",
#     model_name,
#     "--model_nick",
#     model_nick,
#     "--data_dir",
#     data_dir,
#     "--output_dir",
#     os.path.join(output_dir, model_nick),
#     "--cache_dir",
#     os.path.join(output_dir,"cache"),
#     "--overwrite_cache",
#     "--per_device_train_batch_size",
#     train_batch_size,
#     "--per_device_eval_batch_size",
#     eval_batch_size,
#     "--max_seq_len",
#     "64",
#     "--gradient_accumulation_steps",
#     "1",
#     "--num_train_epochs",
#     epochs,
#     "--logging_steps",
#     save_log_steps,
#     "--save_steps",
#     save_log_steps,
#     "--data_parallel",
#     "True",
#     "--meta_task",
#     "paraphrase"
# ])
# tokenizer_para = AutoTokenizer.from_pretrained(model_args.model_name_or_path)
# model_para = AutoModelWithLMHead.from_pretrained(os.path.join(output_dir, model_nick))

In [18]:
data_dir = "../data/pseudo"
# task = "formality"
# task = "abstract+shakespeare+wiki"
# task = "shakespeare"
task = "abstract"
# task = "wiki"
model_name = "t5-small"
# model_nick = "t5_transfer_shakespeare"
# model_nick = "t5_transfer_shakespeare_binary"
# model_nick = "t5_transfer_wiki_binary"
model_nick = "t5_transfer_abstract_binary"
# model_nick = "t5_transfer_macro_binary"
# model_nick = "t5_transfer_formality"
meta_task_type = "binary_single"
meta_task = 'transfer'


output_dir = "../models/"
epochs = "5"
train_batch_size = "16"
eval_batch_size = "16"
save_log_steps = "800"

parser = HfArgumentParser((ModelArguments, DataTrainingArguments, TrainingArguments))
model_args, data_args, training_args = parser.parse_args_into_dataclasses([
    "--model_name_or_path",
    model_name,
    "--model_nick",
    model_nick,
    "--data_dir",
    data_dir,
    "--output_dir",
    os.path.join(output_dir, model_nick),
    "--cache_dir",
    os.path.join(output_dir,"cache"),
    "--overwrite_cache",
    "--per_device_train_batch_size",
    train_batch_size,
    "--per_device_eval_batch_size",
    eval_batch_size,
    "--max_seq_len",
    "64",
    "--gradient_accumulation_steps",
    "1",
    "--num_train_epochs",
    epochs,
    "--logging_steps",
    save_log_steps,
    "--save_steps",
    save_log_steps,
    "--data_parallel",
    "True",
    "--meta_task",
    meta_task,
    "--meta_task_type",
    meta_task_type
])

# Eval
tokenizer_transfer = AutoTokenizer.from_pretrained(model_args.model_name_or_path)
model_transfer = AutoModelWithLMHead.from_pretrained(os.path.join(output_dir, model_nick))

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/t5-small/resolve/main/config.json from cache at /home/vivek/.cache/huggingface/transformers/fe501e8fd6425b8ec93df37767fcce78ce626e34cc5edc859c662350cf712e41.406701565c0afd9899544c1cb8b93185a76f00b31e5ce7f6e18bbaef02241985
Model config T5Config {
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "out

In [19]:
example = "wtf, u think so huh?"
para_example = "paraphrase: "+example
t = tokenizer_para(para_example, return_tensors='pt')
gen = model_para.generate(input_ids= t.input_ids, attention_mask = t.attention_mask, num_return_sequences = 3, num_beams = 12,max_length=70,encoder_no_repeat_ngram_size=5,num_beam_groups=3,diversity_penalty=0.5,)
paraphrases = tokenizer_para.batch_decode(gen, skip_special_tokens=True)
paraphrases

NameError: name 'tokenizer_para' is not defined

In [21]:
paraphrases = ["Machine learning is a new field I tried to explore"]
for p in paraphrases:
    transfer_example = "transfer: "+p
    t = tokenizer_transfer(transfer_example, return_tensors='pt')
    gen = model_transfer.generate(input_ids= t.input_ids, attention_mask = t.attention_mask, max_length=70, 
                                           num_beams=9,
                                           early_stopping=True,
                                           encoder_no_repeat_ngram_size=5,
                                           num_beam_groups=3,
                                           diversity_penalty=0.5,
                                           num_return_sequences=3)
    transfers = tokenizer_transfer.batch_decode(gen, skip_special_tokens=True)
    print(transfers)


['Machine learning (ML) is a novel field that I have tried to explore', 'In this paper, I tried to explore a new field of machine learning', 'In this paper, I tried to explore a new field in machine learning']
